In [2]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import pandas as pd

pd.set_option("display.max_colwidth", 150)

### Parsing xml
parsing xml can be a difficult, I used this video as a referenace
https://www.youtube.com/watch?v=AeRN4zI7Dhk&t=725s

In [3]:
wiki_xml = "./data/leagueoflegends_pages_current.xml"
enc = "utf-8"
parsed_path = "./data/parsed_lol_data.csv"

In [9]:
# get an overview of how xml files are
# and the namespace key for different types of pages

with open(wiki_xml) as f:
    for x in range(70):
        print(f.readline().strip())

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.11/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.11/ http://www.mediawiki.org/xml/export-0.11.xsd" version="0.11" xml:lang="en">
<siteinfo>
<sitename>League of Legends Wiki</sitename>
<dbname>leagueoflegends</dbname>
<base>http://leagueoflegends.fandom.com/wiki/League_of_Legends_Wiki</base>
<generator>MediaWiki 1.39.5</generator>
<case>first-letter</case>
<namespaces>
<namespace key="-2" case="first-letter">Media</namespace>
<namespace key="-1" case="first-letter">Special</namespace>
<namespace key="0" case="first-letter" />
<namespace key="1" case="first-letter">Talk</namespace>
<namespace key="2" case="first-letter">User</namespace>
<namespace key="3" case="first-letter">User talk</namespace>
<namespace key="4" case="first-letter">League of Legends Wiki</namespace>
<namespace key="5" case="first-letter">League of Legends Wiki talk</namespace>
<namespace key="6" 

In [4]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [5]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [6]:
totalCount = 0
title = None
start_time = time.time()

In [10]:
title = ''
id = -99
inrevision = False
ns = 0
text = ''
with codecs.open(parsed_path, "w", enc) as parsed_file:
    articlesWriter = csv.writer(parsed_file, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title', "ns", "text"])
    
    for event, elem in etree.iterparse(wiki_xml, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)
        if event == 'start':
            if tname == 'page': # reset params every page
                title = ''
                id = -99
                inrevision = False
                ns = 0
                text = ''
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
            elif tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision and elem.text!=None:
                id = int(elem.text)
            elif tname == 'ns' and elem.text!=None:
                ns = int(elem.text)
            elif tname == 'text' and elem.text!=None:
                text = elem.text.replace("\n", "").strip()
        elif (event == "end") & (tname == 'text') & (text == "") & (elem.text!=None): # some text only appear in end
            text = elem.text.replace("\n", " ").strip()
        elif tname == 'page':
            totalCount += 1
            articlesWriter.writerow([id, title, ns, text])
            if totalCount > 1 and (totalCount % 100000) == 0:
                print("{:,}".format(totalCount))
        elem.clear()
        
time_took = time.time() - start_time
print(f"Total runtime: {hms_string(time_took)}")

100,000
200,000
300,000
400,000
500,000
600,000
700,000
800,000
900,000
1,000,000
1,100,000
1,200,000
1,300,000
Total runtime: 0:10:04.33


In [4]:
df = pd.read_csv(parsed_path)

In [5]:
df

,id,title,ns,text
0,1297,User:Angela,2,{{wikia:User:Angela}}[[cs:Uživatel:Angela]][[de:Benutzer:Angela]][[fr:Utilisateur:Angela]][[pl:Użytkownik:Angela]]
1,1300,League of Legends Wiki:Administrators,4,#REDIRECT [[League_of_Legends_Wiki:User_Rights]]
2,1452,League of Legends Wiki:Copyrights,4,":'''''Please see [[w:Wikia:Licensing|Wikia:Licensing]] for details of the licensing on this wiki.''''':'''<span style=""color:red; font-size: 150%;..."
3,1460,Category:Browse,14,"This is the top-level or root category.Ideally, every other category should be a subcategory of at least one other, and every article should be in..."
4,1462,Forum:Index,110,Welcome to the {{SITENAME}} forum! This forum can be used by members of this community to discuss topics related to their wiki.:''Notes: You may h...
...,...,...,...,...
1300474,1632022,File:Smolder MMOOOMMMM! HD.png,6,{{Image header/ability}}
1300475,1632023,"File:Smolder Flap, Flap, Flap HD.png",6,{{Image header/ability}}
1300476,1632024,File:Noxus TFT bronze icon.png,6,== Licensing =={{Fairuse}}[[Category:Trait images]]
1300477,1632025,File:Noxus TFT gold icon.png,6,== Licensing =={{Fairuse}}[[Category:Trait images]]


In [6]:
df[df["title"] =="Aatrox/LoL/Audio"]

,id,title,ns,text
1130244,1451514,Aatrox/LoL/Audio,0,{{Section top}} {{LoL navigation}} == Champion Select == ;Pick * {{sm2|Aatrox_Select_SFX.ogg}} '''''Sound Effect''''' * {{sm2|Aatrox_Select.ogg}} ...


In [7]:
df[df["title"].str.contains(".*Yasuo.*", na=False)]

,id,title,ns,text
803876,1059922,File:Yasuo OriginalSquare.png,6,{{Image header/skin}}
804062,1060156,Yasuo/LoL,0,{{Section top}} {{Game banner|Yasuo}} {{LoL navigation}} {{Infobox champion|Yasuo}} {{LoL navigation mobile}} {{Infobox stats|Yasuo}} {{clr}} == A...
804063,1060157,Talk:Yasuo/@comment-9365255-20131115132101,1,OOOOoooooooow my GWWWWWWWWWWaaaaaaaaad!!!11111I bet he will be super cool and overpowered and stuff.Like totally!100% for real!
804064,1060158,Talk:Yasuo/@comment-12617334-20131115160503,1,hello
804095,1060197,Talk:Yasuo/@comment-17834311-20131115193054,1,hi.
...,...,...,...,...
1287362,1618735,File:Yasuo TrueDamageTFTSquare.png,6,{{Image header/skin}}
1287363,1618736,File:Yasuo TrueDamageSquareTFT.png,6,NaN
1287580,1618954,File:Hwei Original FirstEncounterYasuo.ogg,6,== Licensing =={{Fairuse}}[[Category:Hwei voice-overs]]
1288396,1619782,File:Yasuo PROJECTTFTSquare.png,6,{{Image header/skin}}


From looking into the namespaces keys, we can remove lot's of pages based on their namespace,
those pages don't have lore data, most of them are blogs, forums, wiki metadata etc

In [8]:
df = df[~df["ns"].isin([-2,-1,1,2,3,4,5,6,7,8,9,10,11,12,13,15,110,111,114,115,420,421,500,501,502,503,828,829,1200,1201,1202,2000,2001,2002,2300,2301,2302,2900,2901])]

In [9]:
df["ns"].value_counts()

ns
0      19992
14      6842
710        2
Name: count, dtype: int64

In [10]:
df[df["ns"] == 710]

,id,title,ns,text
1151792,1474824,Yuumi 083.ogg.en.srt,710,~Where's my fishy fish fish dish?~ *meow!*
1161125,1485301,Yasuo Original R 0.ogg.en.srt,710,sorye ge ton!


In [11]:
df.to_csv("data/parsed_cleaned_data.csv", index=False)